In [1]:
import os
from pathlib import Path
import random
from tqdm import tqdm

import puppets_data

import numpy as np
import scipy
import networkx as nx

import ConnectionNetworkX as cnx

from pyLDLE2 import datasets
from pyLDLE2 import buml_
from pyLDLE2 import util_


matplotlib.get_backend() =  module://matplotlib_inline.backend_inline


Initializing and pre-processing the puppets data.

In [3]:
p = "/Users/sawyerjack/IdeaProjects/project_data/PuppetsData"
save_dir_root = "/Users/sawyerjack/IdeaProjects/connection-graph-wasserstein/data"

## Just for time and sanity, using n = 100 while I test things.
N_IMAGES = 500

In [3]:
Y, labelsMat, _ = puppets_data.puppets_data(p)

X.shape =  (8100, 100)


Select `N_IMAGES` uniformly at random from the dataset.

In [4]:
X = np.array(Y.copy())
totalImages = X.shape[0]
sampleIndices = random.sample(range(totalImages), N_IMAGES)
X = [ X[i] for i in sampleIndices ]

X = np.array(X)

Initialize a model and use Local PCA at each vertex neighborhood.

In [5]:
N_NEAREST_NEIGHBORS = 15

buml_obj = buml_.BUML(local_opts={'algo':'LPCA', 'k': N_NEAREST_NEIGHBORS},
                      vis_opts={'c': labelsMat[:,0], 'save_dir': save_dir_root},
                      verbose=True, debug=True, exit_at='local_views')

buml_obj.fit(X=X)


local_opts['k_nn0'] = 375 is created.
Options provided:
local_opts:
{
    "Atilde_method": "LDLE_1",
    "N": 100,
    "U_method": "k_nn",
    "algo": "LPCA",
    "alpha": 1,
    "debug": true,
    "delta": 0.9,
    "gl_type": "unnorm",
    "k": 15,
    "k_nn": 49,
    "k_nn0": 375,
    "k_tune": 7,
    "lambda1_decay": 0.75,
    "lambda1_init": 8,
    "lambda1_min": 0.001,
    "max_iter": 300,
    "max_sparsity": 0.9,
    "metric": "euclidean",
    "n_proc": 6,
    "p": 0.99,
    "power": 5,
    "pp_n_thresh": 32,
    "radius": 0.5,
    "reg": 0.0,
    "scale_by": "gamma",
    "tau": 50,
    "to_postprocess": true,
    "verbose": true
}
intermed_opts:
{
    "algo": "best",
    "debug": true,
    "eta_max": 25,
    "eta_min": 5,
    "len_S_thresh": 256,
    "local_algo": "LPCA",
    "metric": "euclidean",
    "n_proc": 6,
    "n_times": 4,
    "verbose": true
}
global_opts:
{
    "add_dim": false,
    "align_transform": "rigid",
    "align_w_parent_only": true,
    "alpha": 0.3,
    "b

Read in the intrinsic dimension of the CNX and then initialize it by symmetrizing the kNN graph.

In [6]:
INTRINSIC_DIMENSION = 2

# U is a directed boolean adjacency matrix, so we symmetrize by adding U.T.
# This weights directed and double edges equally.
puppetGraphAdjacency = buml_obj.LocalViews.U + buml_obj.LocalViews.U.T - scipy.sparse.identity(N_IMAGES, format='lil')

puppetGraph = cnx.ConnectionNetworkX(puppetGraphAdjacency, INTRINSIC_DIMENSION)


Interpolate the local PCA views by registering the overlap of adjacent nodes' neighborhoods, and then using orthogonal procrustes to align the views. This becomes the connection at the corresponding edge.

In [7]:
nRemoteEdges = 0
nEdgesVisited = 0

for i in tqdm(range(N_IMAGES)):
    n_i = nx.neighbors(puppetGraph, i)
    for j in [j for j in n_i if j > i]:

        n_ij = buml_obj.LocalViews.U[i,:].multiply(buml_obj.LocalViews.U[j,:]).nonzero()[1]

        if len(n_ij) >= INTRINSIC_DIMENSION:
            X_Uij_i = buml_obj.LocalViews.local_param_post.eval_({'view_index': i, 'data_mask': n_ij})
            X_Uij_j = buml_obj.LocalViews.local_param_post.eval_({'view_index': j, 'data_mask': n_ij})

            Tij, _ = scipy.linalg.orthogonal_procrustes(X_Uij_i, X_Uij_j)
            nEdgesVisited += 1

            puppetGraph.updateEdgeSignature((i,j), Tij)

        else:
            puppetGraph.removeEdge((i,j))
            nRemoteEdges += 1

print('Proportion of remote edges : ', nRemoteEdges / puppetGraph.nNodes)
print('Proportion of edges visited: ', nEdgesVisited / puppetGraph.nEdges)

100%|██████████| 500/500 [00:19<00:00, 25.08it/s] 

Proportion of remote edges :  0.084
Proportion of edges visited:  1.0


In [8]:
puppetGraph.printConnectionLaplacianEigenvalues()

[3.81224995 3.66945996 3.60606606 3.58116829 3.5630408  3.48305745
 3.39160781 3.29686938 3.0696814  2.8968172 ]
MOST LIKELY INCONSISTENT: |lambda_min| >= 1e-8. 


In [4]:

# puppetGraph = cnx.cnxFromImageDirectory(p, 2, k=15, nImages=500)



X.shape =  (8100, 100)
local_opts['k_nn0'] = 375 is created.
Options provided:
local_opts:
{
    "Atilde_method": "LDLE_1",
    "N": 100,
    "U_method": "k_nn",
    "algo": "LPCA",
    "alpha": 1,
    "debug": true,
    "delta": 0.9,
    "gl_type": "unnorm",
    "k": 15,
    "k_nn": 49,
    "k_nn0": 375,
    "k_tune": 7,
    "lambda1_decay": 0.75,
    "lambda1_init": 8,
    "lambda1_min": 0.001,
    "max_iter": 300,
    "max_sparsity": 0.9,
    "metric": "euclidean",
    "n_proc": 6,
    "p": 0.99,
    "power": 5,
    "pp_n_thresh": 32,
    "radius": 0.5,
    "reg": 0.0,
    "scale_by": "gamma",
    "tau": 50,
    "to_postprocess": true,
    "verbose": true
}
intermed_opts:
{
    "algo": "best",
    "debug": true,
    "eta_max": 25,
    "eta_min": 5,
    "len_S_thresh": 256,
    "local_algo": "LPCA",
    "metric": "euclidean",
    "n_proc": 6,
    "n_times": 4,
    "verbose": true
}
global_opts:
{
    "add_dim": false,
    "align_transform": "rigid",
    "align_w_parent_only": true,
 

100%|██████████| 500/500 [00:19<00:00, 25.33it/s] 


Proportion of edges which were removed due to remoteness:  0.010285179990649837
[3.59804885 3.5413323  3.45396363 3.34897302 3.24961881 3.17085927
 3.07221161 3.06352737 2.90942539 2.78856332]
MOST LIKELY INCONSISTENT: |lambda_min| >= 1e-8. 
